In [0]:
from lxml import etree
from collections import defaultdict

In [2]:
!wget http://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.zip

--2019-05-25 18:37:30--  http://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.zip
Resolving opencorpora.org (opencorpora.org)... 148.251.2.141
Connecting to opencorpora.org (opencorpora.org)|148.251.2.141|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2190887 (2.1M) [application/zip]
Saving to: ‘annot.opcorpora.no_ambig_strict.xml.zip’

annot.opcorpora.no_ 100%[===================>]   2.09M  2.02MB/s    in 1.0s    

2019-05-25 18:37:31 (2.02 MB/s) - ‘annot.opcorpora.no_ambig_strict.xml.zip’ saved [2190887/2190887]



In [3]:
!unzip annot.opcorpora.no_ambig_strict.xml.zip

Archive:  annot.opcorpora.no_ambig_strict.xml.zip
  inflating: annot.opcorpora.no_ambig_strict.xml  


In [0]:
open_corpora = etree.fromstring(open('annot.opcorpora.no_ambig_strict.xml', 'rb').read())
corpus = open('corpus_train.txt', 'w', encoding = 'utf-8')
vocab = defaultdict(set)
tags = set()

for sentence in open_corpora.xpath('//tokens'):
    length = len(sentence.xpath('token'))
    ended = False
    for i,token in enumerate(sentence.xpath('token')):
        word = token.xpath('@text')
        gram_info = token.xpath('tfr/v/l/g/@v')
        
        if (i+1)==length and gram_info[0] == 'PNCT':
            gram_info = ['SENT']
            ended = True
        
            
        corpus.write(word[0] + '\t' + ','.join(gram_info) + '\n')
        lemma = token.xpath('tfr/v/l/@t')[0]
        vocab[word[0].lower()].add((','.join(gram_info), lemma.lower()))
        tags.add(','.join(gram_info))
    
    if not ended:
        corpus.write('.\tSENT\n')
f = open('lexicon.txt', 'w', encoding = 'utf-8')

for word in vocab:
    f.write(word + '\t')
    f.write('\t'.join([' '.join(pair) for pair in vocab[word]]))
    f.write('\n')
# f.write('SENT\tSENT .')
f.close()
f = open('open_class.txt', 'w')

f.write('\n'.join([tag for tag in tags if 'NOUN' in tag or 'VERB' in tag or 'ADJF' in tag]))
f.close()

In [0]:
import keras
from collections import Counter

In [0]:
corpus = []
vocab = defaultdict(set)
tags = set()

for sentence in open_corpora.xpath('//tokens'):
    sent = []
    for token in sentence.xpath('token'):
        word = token.xpath('@text')
        gram_info = token.xpath('tfr/v/l/g/@v')
        
        
            
        sent.append(word + gram_info)
    corpus.append(sent)

In [49]:
corpus[0]

[['«', 'PNCT'],
 ['Школа', 'NOUN', 'inan', 'femn', 'sing', 'nomn'],
 ['злословия', 'NOUN', 'inan', 'neut', 'sing', 'gent'],
 ['»', 'PNCT'],
 ['учит', 'VERB', 'impf', 'tran', 'sing', '3per', 'pres', 'indc'],
 ['прикусить', 'INFN', 'perf', 'tran'],
 ['язык', 'NOUN', 'inan', 'masc', 'sing', 'accs']]

In [50]:
metric_corpus = corpus[int(len(corpus)*0.97):]
corpus = corpus[:int(len(corpus)*0.97)]
len(corpus), len(metric_corpus)

(10272, 318)

In [51]:
vocab = Counter()
poses = Counter()

for sent in corpus:
  for word, pos, *tags in sent:
    vocab[word.lower()] += 1
    poses[pos] += 1
vocab = {word for word,c in vocab.most_common() if c > 3}

len(vocab), len(poses)

(2122, 22)

Сами слова соберем таким образом

In [0]:
id2word = {i+2:word for i, word in enumerate(vocab)}
id2word[0] = '<PAD>'
id2word[1] = '<UNK>'
word2id = {word:i for i, word in id2word.items()}

Теперь теги и признаки:
- одушивлённость
-падеж
- число
- переходность
-вид

In [0]:
id2pos = {i+1:pos for i, pos in enumerate(poses)}
id2pos[0] = '<PAD>'
pos2id = {pos:i for i, pos in id2pos.items()}

In [0]:
id2anim = {1:'NONE', 2:'anim', 3:'inan'}
id2anim[0] = '<PAD>'
anim2id = {tag:i for i, tag in id2anim.items()}

In [0]:
id2case = {1:'NONE', 2:'nomn', 3:'gent', 4:'datv', 
           5:'accs', 6:'ablt', 7:'loct'}
id2case[0] = '<PAD>'
case2id = {tag:i for i, tag in id2case.items()}

In [0]:
id2num = {1:'NONE', 2:'sing', 3:'plur'}
id2num[0] = '<PAD>'
num2id = {tag:i for i, tag in id2num.items()}

In [0]:
id2tran = {1:'NONE', 2:'tran', 3:'intr'}
id2tran[0] = '<PAD>'
tran2id = {tag:i for i, tag in id2tran.items()}

In [0]:
id2asp = {1:'NONE', 2:'perf', 3:'impf'}
id2asp[0] = '<PAD>'
asp2id = {tag:i for i, tag in id2tran.items()}

In [0]:
sents_ids = []
poses_ids = []
anim_ids = []
case_ids = []
num_ids = []
tran_ids = []
asp_ids = []

for sent in corpus:
    sents_ids.append([word2id.get(word.lower(), 1) for word, *_ in sent])
    poses_ids.append([pos2id[tag] for word, tag, *_ in sent])
    
    anim_ids_sent = []
    case_ids_sent = []
    num_ids_sent = []
    tran_ids_sent = []   
    asp_ids_sent = []
    
    for word, pos, *tags in sent:
      anim_tag, case_tag, num_tag, tran_tag, asp_tag = 1, 1, 1, 1, 1
      for tag in tags:
        
        if tag in anim2id:
          anim_tag = anim2id[tag]
        if tag in case2id:
          case_tag = case2id[tag]
        if tag in num2id:
          num_tag = num2id[tag]
        if tag in tran2id:
          tran_tag = tran2id[tag]
        if tag in asp2id:
          asp_tag = asp2id[tag]
      
      if pos == "VERB":
        anim_tag = 1
        case_tag = 1
      elif pos == "NOUN":
        tran_tag = 1
        asp_tag = 1
      elif pos == "ADV":
        anim_tag = 1
        case_tag = 1 
        num_tag = 1 
        tran_tag =1 
        asp_tag = 1
      elif pos == "ADJF" or pos == "ADJS":
        tran_tag = 1
        anim_tag = 1
        asp_tag = 1
      elif pos == "PRTF" or pos == "PRTS":
        anim_tag = 1
      elif pos == "PRCL" or pos == "PRCL" or pos == "INTJ" or pos == "CONJ" or pos == "PREP":
        anim_tag = 1
        case_tag = 1 
        num_tag = 1 
        tran_tag =1 
        asp_tag = 1
      
      anim_ids_sent.append(anim_tag)
      case_ids_sent.append(case_tag)
      num_ids_sent.append(num_tag)
      tran_ids_sent.append(tran_tag)
      asp_ids_sent.append(asp_tag)
    anim_ids.append(anim_ids_sent)
    case_ids.append(case_ids_sent)
    num_ids.append(num_ids_sent)
    tran_ids.append(tran_ids_sent)
    asp_ids.append(asp_ids_sent)

In [0]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, LSTM, Input, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam

In [0]:
MAX_LENGTH = 10
sents_ids_padded = pad_sequences(sents_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')
poses_ids_padded = pad_sequences(poses_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')
anim_ids_padded = pad_sequences(anim_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')
case_ids_padded = pad_sequences(case_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')
num_ids_padded = pad_sequences(num_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')
tran_ids_padded = pad_sequences(tran_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')
asp_ids_padded = pad_sequences(asp_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')

In [0]:
train_index, test_index = train_test_split(list(range(sents_ids_padded.shape[0])),test_size=0.2)

In [0]:
train_sents, test_sents = sents_ids_padded[train_index], sents_ids_padded[test_index]
train_pos, test_pos = poses_ids_padded[train_index], poses_ids_padded[test_index]
train_anim, test_anim = anim_ids_padded[train_index], anim_ids_padded[test_index]
train_case, test_case = case_ids_padded[train_index], case_ids_padded[test_index]
train_num, test_num = num_ids_padded[train_index], num_ids_padded[test_index]
train_tran, test_tran = tran_ids_padded[train_index], tran_ids_padded[test_index]
train_asp, test_asp = asp_ids_padded[train_index], asp_ids_padded[test_index]

In [0]:
inp = Input(shape=(MAX_LENGTH, ))
x = Embedding(len(word2id), 8)(inp)

x = LSTM(20, return_sequences=True)(x)

pos = TimeDistributed(Dense(len(pos2id), activation='softmax'))(x)
anim = TimeDistributed(Dense(len(anim2id), activation='softmax'))(x)
case = TimeDistributed(Dense(len(case2id), activation='softmax'))(x)
num = TimeDistributed(Dense(len(num2id), activation='softmax'))(x)
tran = TimeDistributed(Dense(len(tran2id), activation='softmax'))(x)
asp = TimeDistributed(Dense(len(asp2id), activation='softmax'))(x)

 
model = Model(inputs=inp, outputs=[pos, anim, case, num, tran, asp], 
           )
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
from keras.utils import to_categorical

In [66]:
model.fit(x=train_sents, 
        y=[to_categorical(train_pos), to_categorical(train_anim), to_categorical(train_case), to_categorical(train_num), to_categorical(train_tran), to_categorical(train_asp)], batch_size=128, epochs=150, validation_split=0.2)

Train on 6573 samples, validate on 1644 samples
Epoch 1/150
6573/6573 [==============================] - 3s 522us/step - loss: 10.2999 - time_distributed_13_loss: 3.0265 - time_distributed_14_loss: 1.3168 - time_distributed_15_loss: 2.0396 - time_distributed_16_loss: 1.3327 - time_distributed_17_loss: 1.2942 - time_distributed_18_loss: 1.2901 - time_distributed_13_acc: 0.1812 - time_distributed_14_acc: 0.4122 - time_distributed_15_acc: 0.1736 - time_distributed_16_acc: 0.3841 - time_distributed_17_acc: 0.6698 - time_distributed_18_acc: 0.7894 - val_loss: 9.1931 - val_time_distributed_13_loss: 2.7369 - val_time_distributed_14_loss: 1.1519 - val_time_distributed_15_loss: 1.9316 - val_time_distributed_16_loss: 1.2023 - val_time_distributed_17_loss: 1.0959 - val_time_distributed_18_loss: 1.0745 - val_time_distributed_13_acc: 0.1443 - val_time_distributed_14_acc: 0.4282 - val_time_distributed_15_acc: 0.5377 - val_time_distributed_16_acc: 0.4282 - val_time_distributed_17_acc: 0.5676 - val_ti

Ура! нейросеть обучилась! И в колабе это занялр значительно меньше времени!!!!

In [0]:
def predicto(sent):
  d = {} 
  
  test_sample = []
  test_sample.extend([word[0] for word in sent])
  test_samples=[test_sample]
  test_samples_X = []
  
  for sent in test_samples:
    sent_ids=[]
    for word in sent:
      if word not in word2id:
        pass
      sent_ids.append(word2id.get(word.lower(), 1))
    test_samples_X.append(sent_ids)
  test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
  predictions = model.predict(test_samples_X)
  
  for i, idx in enumerate(predictions[0].argmax(axis=2)[0]):
    if i > (len(test_samples[0])-1):
      break
    d[i] = []
    d[i].append(id2pos[idx])
  
  for i, idx in enumerate(predictions[1].argmax(axis=2)[0]):
    if i > (len(test_samples[0])-1):
      break
    
    if d[i][0] == 'VERB' or d[i][0] == 'PNCT' or d[i][0]=='ADVB' or d[i][0] == 'ADJF' or d[i][0] == "PREP":
      d[i].append('NONE')
    else:
      d[i].append(id2anim[idx])  


  for i, idx in enumerate(predictions[2].argmax(axis=2)[0]):
    if i > (len(test_samples[0])-1):
      break    
    if d[i][0] == 'VERB' or d[i][0] == 'PNCT' or d[i][0] == 'ADVB' or d[i][0] == "PREP":
      pass
    else:  
      d[i].append(id2case[idx])
  
  for i, idx in enumerate(predictions[3].argmax(axis=2)[0]):
    if i > (len(test_samples[0])-1):
      break
    if d[i][0] == 'PNCT' or d[i][0] == 'ADVB' or d[i][0] == "PREP":
      pass
    else:
      d[i].append(id2num[idx])
  
  for i, idx in enumerate(predictions[4].argmax(axis=2)[0]):
    if i > (len(test_samples[0])-1):
      break
    if d[i][0] != 'VERB':
      pass
    else:
      d[i].append(id2tran[idx])
    
  for i, idx in enumerate(predictions[5].argmax(axis=2)[0]):
    if i > (len(test_samples[0])-1):
      break    
    if d[i][0] == 'PNCT' or d[i][0] == 'ADVB' or d[i][0] == "PREP" or d[i][0] == 'ADJF' or d[i][0] == 'NOUN' :
      pass
    else:  
      d[i].append(id2asp[idx])
    
    for key in d.keys():      
      while 'NONE' in d[key]:
        d[key].remove('NONE')    
    
  return d   

In [68]:
import numpy as np

metrics = []
count_full = 0
mistakes = []

for sent in metric_corpus:

  sent_metrics = []
  sent_real = []
  words = []
  if len(sent) > 10:
    sent = sent[:10]
  for word in sent:
    sent_real.append(word[1:])
    words.append(word[0])
  sent_pred = predicto(sent)
  
  
  for i in range(0, len(sent_real)):
    set_real = set(sent_real[i])     
    set_pred = set(sent_pred[i])
    metric = len(set_real&set_pred) / len(set_pred|set_real)
    sent_metrics.append(metric)
  
  if np.mean(sent_metrics) == 1:
    count_full += 1
  if np.mean(sent_metrics) < 0.5:
    mistakes.append(words)
  
  if len(sent_metrics)==0:
    continue
  metrics.append(np.mean(sent_metrics))
  
print(len(metrics))
print(len(metric_corpus))

318
318


Посчитаем Метрику Жаккара

In [69]:
print(np.mean(metrics))

0.7032240840372953


На этом всё!